You are evaluating a candidate sentiment model to replace a production baseline. Your goal is to determine whether this model should ship.

“Ship” means: we would choose the candidate model over the baseline for deployment based on the evidence you collect.

### Step 1 - Install the required dependencies, set up W&B and make sure the python version is 3.10 and above

In [1]:
!pip install -q wandb datasets transformers evaluate tqdm emoji regex pandas pyarrow scikit-learn nbformat torch

import os
os.environ["TOKENIZERS_PARALLELISM"] = "false"

import numpy as np
import pandas as pd
import emoji
import wandb

from datasets import load_dataset
from transformers import pipeline

SEED = 42
np.random.seed(SEED)

/opt/anaconda3/envs/Nutcracker/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
import os, wandb
wandb.login()

wandb: [wandb.login()] Loaded credentials for https://api.wandb.ai from /Users/improvise/.netrc.
wandb: Currently logged in as: eames_shi (eames_shi-carnegie-mellon-university) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


True

In [3]:
!python --version

Python 3.12.12


In [4]:
#imports and config:
import re, regex, emoji
import pandas as pd
import numpy as np
import tqdm

import wandb
from datasets import load_dataset
from transformers import pipeline
import evaluate


# WANDB CONFIG
PROJECT = "mlip-lab4-slices-2026"
ENTITY = None
RUN_NAME = "baseline_vs_candidate"


In [5]:
# Models to compare
MODELS = {
    "baseline_model": "cardiffnlp/twitter-roberta-base-sentiment-latest",
    "candidate_model":    "LYTinn/finetuning-sentiment-model-tweet-gpt2",
}

In [6]:
# Label normalization for tweet_eval (0/1/2 -> string labels)
ID2LABEL = {0: "negative", 1: "neutral", 2: "positive"}

# Many HF sentiment models output labels like LABEL_0 / LABEL_1 / LABEL_2
HF_LABEL_MAP = {"LABEL_0": "negative", "LABEL_1": "neutral", "LABEL_2": "positive"}

USE_HF_DATASET = True  # set False to use tweets.csv fallback

### Step 2 - Load a dataset from Hugging Face

In [7]:
if USE_HF_DATASET:
    ds = load_dataset("cardiffnlp/tweet_eval", "sentiment")
    df = pd.DataFrame(ds["test"]).head(500).copy()
    df["label"] = df["label"].map(ID2LABEL)
else:
    df = pd.read_csv("tweets.csv")
    # Ensure it has 'text' and 'label' columns
    df = df.rename(columns={c: c.strip() for c in df.columns})
    assert {"text","label"}.issubset(df.columns), "tweets.csv must include text,label"
    df["label"] = df["label"].astype(str).str.lower()

df = df[["text","label"]].dropna().reset_index(drop=True)
df.head(3)


Generating validation split: 100%|██████████| 2000/2000 [00:00<00:00, 1176522.86 examples/s]


,text,label
0,@user @user what do these '1/2 naked pics' hav...,neutral
1,OH: “I had a blue penis while I was this” [pla...,neutral
2,"@user @user That's coming, but I think the vic...",neutral


### Step 3 - Define Failure-Relevant Metadata

#TODO:
In this step, you will create **at least 5** metadata columns that help you slice and analyze model behavior in Weights & Biases (W&B).
These metadata columns should **capture meaningful properties of the data or model behavior that may influence performance**. You can define them using:

1. Value matching (e.g., tweets containing hashtags or mentions)
2. Regex patterns (e.g., negation words, strong sentiment terms like love or hate)
3. Heuristics (e.g., emoji count, all-caps text, tweet length buckets)

Each metadata column should correspond to a potential hypothesis about when or why a model might succeed or fail.
These columns will be propagated through inference and included in the final predictions_table logged to W&B.

After inference, your W&B table (df_long) will contain:
- The original tweet text
- Ground-truth sentiment labels
- Model predictions and confidence scores
- All metadata columns you defined for slicing

You will use these metadata fields in the W&B UI (via the ➕ Filter option) to:
- Create slices of the data
- Compare model behavior across slices
- Identify patterns, weaknesses, or regressions that are not visible in overall accuracy

In [17]:
# Step 3 – Add slicing metadata (text-only)

# TODO: add your own hypothesis-driven metadata here. 
# Here are examples of the kinds of metadata columns you can add & analyse.
# Categories you can explore are: Linguistic, Emotional/semantic, Model-behavioral. Do not reuse the ones given below.
def count_emojis(text: str) -> int:
    return sum(ch in emoji.EMOJI_DATA for ch in str(text))

df["emoji_count"] = df["text"].apply(count_emojis).astype(int)
df["has_hashtag"] = df["text"].str.contains(r"#\w+", regex=True)
df["has_mention"] = df["text"].str.contains(r"@\w+", regex=True)
df["has_negation"] = df["text"].str.contains(r"\b(not|never|no)\b", regex=True)
df["length_bucket"] = pd.cut(
    df["text"].str.len(),
    bins=[0, 50, 100, 200, 1000, 10_000],
    labels=["0-50", "51-100", "101-200", "201-1000", "1001+"],
    include_lowest=True
).astype(str)

df["has_strong_emotion"] = df["text"].str.contains(r"\b(much|very|too|deeply)\b", regex = True) # strong emotions helps model decide
df["has_turning"] = df["text"].str.contains(r"\b(but|however|though|although|while|whereas)\b", regex=True) # constrast words contain complex emotions
df["has_affirmation"] = df["text"].str.contains(r"\b(yes|love|sure|great|)\b", regex=True)
df["avg_length_per_sentence"] = df["text"].apply( 
    lambda text: np.mean([len(s) for s in text.split(".") if s.strip()])                                             
)
df["has_question"] = df["text"].str.contains(r"\?", regex=True)
df["has_all_caps_word"] = df["text"].str.contains(r"\b[A-Z]{2,}\b", regex=True)


# Example slice definitions (you'll create more later)
def get_slices(df_any: pd.DataFrame):
    return {
        # "emoji_gt3":            df_any["emoji_count"] > 3
        # , "has_negation":       df_any["has_negation"] == True
        # , "has_hashtag":        df_any["has_hashtag"] == True
        "has_strong_emotion": df_any["has_strong_emotion"] == True
        , "has_turning":        df_any["has_turning"] == True
        , "has_question":       df_any["has_question"] == True 
    }

/var/folders/kd/t2wnd4h911v09vtv543hpkg80000gn/T/ipykernel_28759/444132091.py:12: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  df["has_negation"] = df["text"].str.contains(r"\b(not|never|no)\b", regex=True)
/var/folders/kd/t2wnd4h911v09vtv543hpkg80000gn/T/ipykernel_28759/444132091.py:20: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  df["has_strong_emotion"] = df["text"].str.contains(r"\b(much|very|too|deeply)\b", regex = True) # strong emotions helps model decide
/var/folders/kd/t2wnd4h911v09vtv543hpkg80000gn/T/ipykernel_28759/444132091.py:21: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  df["has_turning"] = df["text"].str.contains(r"\b(but|however|though|although|while|whereas)\b", regex=True) # constrast words contain co

In [18]:
# Transformers requires a backend (PyTorch/TensorFlow/Flax). We'll use PyTorch.
try:
    import torch, transformers, sys
    print("torch:", torch.__version__)
    print("transformers:", transformers.__version__)
    print("CUDA available:", torch.cuda.is_available())
    print("MPS available: ", torch.mps.is_available())
    print("Python:", sys.executable)
except Exception as e:
    raise RuntimeError("Install PyTorch before proceeding: pip install torch torchvision torchaudio") from e

torch: 2.9.0
transformers: 4.57.3
CUDA available: False
MPS available:  True
Python: /opt/anaconda3/envs/Nutcracker/bin/python


###  Step 4 – Run Inference (Two Models)

In this step, you'll use two HuggingFace sentiment analysis models to run inference on your dataset:

In [19]:
from tqdm.auto import tqdm

def run_pipeline(model_id: str, texts: list[str]):
    clf = pipeline(
        "text-classification",
        model=model_id,
        truncation=True,
        max_length=128,     # avoid truncation warnings
        framework="pt",
        device= "mps"           # CPU
    )
    # (Optional) sanity check label mapping for this model
    # print(model_id, clf.model.config.id2label)

    preds, confs = [], []
    for t in tqdm(texts, desc=f"Infer: {model_id}"):
        out = clf(t)[0]
        lbl = HF_LABEL_MAP.get(out["label"], out["label"])
        preds.append(lbl)
        confs.append(float(out["score"]))
    return preds, confs

pred_frames = []
texts = df["text"].tolist()

for model_name, model_id in MODELS.items():
    yhat, conf = run_pipeline(model_id, texts)
    tmp = df.copy()
    tmp["model"] = model_name
    tmp["pred"] = yhat
    tmp["conf"] = conf
    pred_frames.append(tmp)

df_long = pd.concat(pred_frames, ignore_index=True)

# Add a stable example id so reshaping won't silently drop duplicates
df_long["ex_id"] = df_long.groupby(["text", "label"]).ngroup()

df_long.head(5)

Some weights of the model checkpoint at cardiffnlp/twitter-roberta-base-sentiment-latest were not used when initializing RobertaForSequenceClassification: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Device set to use mps
Infer: cardiffnlp/twitter-roberta-base-sentiment-latest: 100%|██████████| 500/500 [00:18<00:00, 26.87it/s]
Device set to use mps
Infer: LYTinn/finetuning-sentiment-model-tweet-gpt2: 100%|██████████| 500/500 [00:15<00:00, 31.82it/s]


,text,label,emoji_count,has_hashtag,has_mention,has_negation,length_bucket,has_strong_emotion,has_turning,has_affirmation,avg_length_per_sentence,has_question,has_all_caps_word,model,pred,conf,ex_id
0,@user @user what do these '1/2 naked pics' hav...,neutral,0,False,True,True,51-100,False,False,True,95.0,True,False,baseline_model,negative,0.804726,113
1,OH: “I had a blue penis while I was this” [pla...,neutral,0,False,False,False,51-100,False,True,True,72.0,False,True,baseline_model,neutral,0.866949,363
2,"@user @user That's coming, but I think the vic...",neutral,0,False,True,False,51-100,False,True,True,86.0,False,False,baseline_model,neutral,0.763725,102
3,I think I may be finally in with the in crowd ...,positive,0,True,True,False,51-100,False,False,True,83.0,False,False,baseline_model,positive,0.774047,305
4,"@user Wow,first Hugo Chavez and now Fidel Cast...",negative,0,False,True,False,101-200,False,False,True,65.5,False,False,baseline_model,neutral,0.416397,160


In [20]:
# Step 4.5 – Wide-format Table for Model Comparison (Optional but recommended)
# One row per tweet, with baseline + candidate predictions in columns
# TODO: Replace with your metadata
df_wide = df_long.pivot_table(
    index=[
        "ex_id", "text", "label",
        "has_strong_emotion", "has_turning", "has_affirmation", "avg_length_per_sentence", "has_question", "has_all_caps_word"
    ],
    columns="model",
    values=["pred", "conf"],
    aggfunc="first"
).reset_index()

# Flatten column names (e.g., pred_baseline_model, conf_candidate_model)
df_wide.columns = ["_".join([c for c in col if c]).strip("_") for col in df_wide.columns]

df_wide.head(5)

,ex_id,text,label,has_strong_emotion,has_turning,has_affirmation,avg_length_per_sentence,has_question,has_all_caps_word,conf_baseline_model,conf_candidate_model,pred_baseline_model,pred_candidate_model
0,0,"""Fatty Kim The Third"" 😭😭😭",neutral,False,False,True,25.000000,False,False,0.486252,0.978987,neutral,neutral
1,1,"""Focusing on [alt rightists'] respectability.....",neutral,False,False,True,31.666667,False,False,0.573572,0.999728,negative,neutral
2,2,"""Kim Fatty the Third""",negative,False,False,True,21.000000,False,False,0.849732,0.937709,neutral,neutral
3,3,"""We have lost everything"": Syrians return to r...",neutral,False,False,True,45.500000,False,True,0.751955,0.994244,negative,positive
4,4,"""who's the most wiped out white boy? Zac Efron...",neutral,False,False,True,59.000000,True,False,0.561233,0.906540,neutral,positive


### Step 5: Compute Metrics (Accuracy + Slice Accuracy + Regression)

In [21]:
# TODO: Edit to work for your slices

#compute metrics model-wise
from sklearn.metrics import accuracy_score

def compute_accuracy(y_true, y_pred):
    return accuracy_score(list(y_true), list(y_pred))

# Overall accuracy by model (df_long: one row per (tweet, model))
overall = df_long.groupby("model").apply(
    lambda g: compute_accuracy(g["label"], g["pred"]),
    include_groups=False
)

# Slice accuracy table (uses df_long masks)
slice_table = wandb.Table(columns=["slice", "model", "accuracy"])
slice_metrics = {}

for slice_name, mask in get_slices(df_long).items():
    slice_metrics[slice_name] = {}
    for model_name, g in df_long[mask].groupby("model"):
        acc = float(compute_accuracy(g["label"], g["pred"]))
        slice_table.add_data(slice_name, model_name, acc)
        slice_metrics[slice_name][model_name] = acc

In [22]:
# TODO: Edit to work for your slices


# Regression-aware evaluation (df_eval: one row per tweet, both model outputs) 
# A regression is when the candidate gets something wrong that the baseline got right.
BASELINE = "baseline_model"
CANDIDATE = "candidate_model"

# Ensure ex_id exists (safe even if it already exists)
df_long = df_long.copy()
if "ex_id" not in df_long.columns:
    df_long["ex_id"] = df_long.groupby(["text", "label"]).ngroup()

# Build df_eval with metadata carried through
df_eval = (
    df_long.pivot_table(
        index=[
            "ex_id", "text", "label",
            "has_strong_emotion", "has_turning", "has_affirmation", "avg_length_per_sentence", "has_question", "has_all_caps_word"
        ],
        columns="model",
        values=["pred", "conf"],
        aggfunc="first"
    )
    .reset_index()
)

# Flatten column names (pred_baseline_model, conf_candidate_model, etc.)
df_eval.columns = ["_".join([c for c in col if c]).strip("_") for col in df_eval.columns]

# Correctness flags
df_eval["baseline_correct"]  = df_eval[f"pred_{BASELINE}"] == df_eval["label"]
df_eval["candidate_correct"] = df_eval[f"pred_{CANDIDATE}"] == df_eval["label"]

# Regression / improvement flags
df_eval["regressed"]   = df_eval["baseline_correct"] & ~df_eval["candidate_correct"]
df_eval["improved"]    = ~df_eval["baseline_correct"] & df_eval["candidate_correct"]
df_eval["both_wrong"]  = ~df_eval["baseline_correct"] & ~df_eval["candidate_correct"]
df_eval["both_correct"]= df_eval["baseline_correct"] & df_eval["candidate_correct"]

# Confidence-conditional regression (candidate is confident AND worse than baseline)
df_eval["confident_regression"] = df_eval["regressed"] & (df_eval[f"conf_{CANDIDATE}"] >= 0.8)

# Global regression metrics
regression_rate = float(df_eval["regressed"].mean())
improvement_rate = float(df_eval["improved"].mean())
conf_reg_rate = float(df_eval["confident_regression"].mean())

print("Regression rate:", regression_rate)
print("Improvement rate:", improvement_rate)
print("Confident regression rate:", conf_reg_rate)

Regression rate: 0.408
Improvement rate: 0.108
Confident regression rate: 0.382


In [23]:
# TODO: Edit to work with your slices

# Define slices on df_eval (must use columns that exist in df_eval)
def get_slices_eval(df_any):
    return {
        "has_strong_emotion": df_any["has_strong_emotion"] == True
        , "has_turning":        df_any["has_turning"] == True
        , "has_question":       df_any["has_question"] == True 
    }

# Slice-level regression metrics table
reg_table = wandb.Table(columns=["slice", "metric", "value"])
reg_metrics = {}

for slice_name, mask in get_slices_eval(df_eval).items():
    g = df_eval[mask]
    if len(g) == 0:
        continue

    reg = float(g["regressed"].mean())
    imp = float(g["improved"].mean())
    conf_reg = float(g["confident_regression"].mean())

    reg_table.add_data(slice_name, "regression_rate", reg)
    reg_table.add_data(slice_name, "improvement_rate", imp)
    reg_table.add_data(slice_name, "confident_regression_rate", conf_reg)

    reg_metrics[slice_name] = {
        "regression_rate": reg,
        "improvement_rate": imp,
        "conf_reg_rate": conf_reg
    }



# Step 6 — #TODO: Log to W&B & Analyse Slices
# (Make sure PROJECT/ENTITY/RUN_NAME exist from Step 1)

In [24]:
# Step 6: Log to W&B

PROJECT = "mlip-lab4-slices-2026"
ENTITY = None
RUN_NAME = "baseline_vs_candidate"
run = wandb.init(project=PROJECT, entity=ENTITY, name=RUN_NAME)
wandb.log({"predictions_table": wandb.Table(dataframe=df_long)})
wandb.log({"slice_metrics": slice_table})
wandb.log({"regression_metrics": reg_table})
wandb.log({
    "df_eval": wandb.Table(dataframe=df_eval)
})
for model_name, acc in overall.items():
    wandb.summary[f"{model_name}_accuracy"] = float(acc)
wandb.summary["regression_rate"] = regression_rate
wandb.summary["improvement_rate"] = improvement_rate
wandb.summary["confident_regression_rate"] = conf_reg_rate

print("W&B run URL:", run.get_url())
run.finish()

wandb: WARNING The get_url method is deprecated and will be removed in a future release. Please use `run.url` instead.


W&B run URL: https://wandb.ai/eames_shi-carnegie-mellon-university/mlip-lab4-slices-2026/runs/rh5oa3uc


baseline_model_accuracy,0.698
candidate_model_accuracy,0.398
confident_regression_rate,0.382
improvement_rate,0.108
regression_rate,0.408


### Instructions: Exploring Slice-Based Evaluation in W&B

# Purpose
In this lab, you are evaluating a candidate sentiment model to decide whether it should replace an existing baseline (production) model.
You have already:
  - run both models on the same dataset
  - logged predictions, confidence scores, and metadata to W&B
  - created metadata that allows you to slice the data
The most important goal is to understand when and why models behave differently.
Overall accuracy alone is often misleading.

# What to do in W&B
1. Open your W&B run
  - Click the project link and open the latest run.
2. Explore the predictions table
  - Go to the Tables tab and open predictions_table.
  - Each row is one tweet × one model.
3. Create and analyze slices (most important)
  - Use filters to create meaningful slices 
    (e.g., negation, emojis, hashtags, long tweets).
  - For each slice:
    - Compare baseline vs candidate performance.
    - Compare slice accuracy to overall accuracy.
    - Inspect a few misclassified examples to identify patterns.
4. Visualize slice performance
  - Open slice_metrics.
  - Create bar charts comparing baseline vs candidate accuracy for at least two slices.
5. Discuss your findings with the TA
  - Explain why slicing reveals issues that overall accuracy hides.
  - Say whether the candidate model should be deployed and why.


In [25]:
# Students: replace the placeholders below with 1–2 sentence insights
#TODO: Replace this with 1-2 sentence takeaways for each slice.
saved_slice_notes = ["If a tweet has turning words, the baseline model can perform really well. Strong emotion words tend to mess up the accuracy for both baseline and candidate model."]
pd.DataFrame(saved_slice_notes)

,0
0,"If a tweet has turning words, the baseline mod..."


### Step 7 - Targeted stress testing with LLMs

TODO: 
In this step, you will use a Large Language Model (LLM) to generate test cases that specifically target a weakness you observed during slicing.

What to do:
1. Choose one slice where you noticed poor performance, regressions, or surprising behavior.
2. Write a short hypothesis (1–2 sentences) explaining why the model might struggle on this slice. Example:
“The model struggles with tweets that use slang and sarcasm.”
3. Use an LLM to generate 10 test cases designed to test this hypothesis.
These can include:
    - subtle or ambiguous cases
    - difficult or adversarial cases
    - small wording changes that affect sentiment
4. Re-run both models on the generated test cases (helper script given below.)
5. Briefly describe what you observed to the TA:
    - Did the same failures appear again?
    - notice any new failure patterns?
    - would this affect your confidence in deploying the model?

Your input can be in the following format:

> Examples:
> - @user @user That’s coming, but I think the victims are going to be Medicaid recipients.
> - I think I may be finally in with the in crowd #mannequinchallenge  #grads2014 @user
> 
> Generate more tweets using slangs.

Use our provided GPTs to start the task: [llm-based-test-case-generator](https://chatgpt.com/g/g-982cylVn2-llm-based-test-case-generator). If you do not have access to GPTs, use the plain ChatGPT or other LLM providers you have access to instead.

In [26]:
# TODO: Paste your 10 generated tweets here:

generated_slice_description = "The model seems to struggle with strong-emotion based tweets. Created tweets that contains strong-emotion indicators, such as `very`, `much`, `deeply`, `too`"

generated_cases = [
    "I’m very tired of pretending everything is fine."
    , "This means so much to me — I can’t even explain it."
    , "That moment was just too painful to revisit."
    , "I’m deeply grateful for everyone who stood by me."
    , "I care too much and it always ends up hurting."
    , "Feeling very overwhelmed right now. Everything is hitting at once."
    , "I miss you so much it physically hurts."
    , "Deeply disappointed but not surprised."
    , "This is too good to be true — waiting for the catch."
    , "Very proud of how far I’ve come, even if no one else sees it."
]

In [27]:
#Helper code to run models on synthetic test cases:

def run_on_generated_tests(texts, models=MODELS):
    rows = []
    for model_name, model_id in models.items():
        clf = pipeline(
            "text-classification",
            model=model_id,
            truncation=True,
            framework="pt",
            device= "mps"
        )
        for t in texts:
            out = clf(t)[0]
            rows.append({
                "text": t,
                "model": model_name,
                "pred": HF_LABEL_MAP.get(out["label"], out["label"]),
                "conf": float(out["score"])
            })
    return pd.DataFrame(rows)


In [28]:
generated_df = run_on_generated_tests(generated_cases)
generated_df

Some weights of the model checkpoint at cardiffnlp/twitter-roberta-base-sentiment-latest were not used when initializing RobertaForSequenceClassification: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Device set to use mps
Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.
Device set to use mps


,text,model,pred,conf
0,I’m very tired of pretending everything is fine.,baseline_model,negative,0.837370
1,This means so much to me — I can’t even explai...,baseline_model,positive,0.978007
2,That moment was just too painful to revisit.,baseline_model,negative,0.914086
3,I’m deeply grateful for everyone who stood by me.,baseline_model,positive,0.980864
4,I care too much and it always ends up hurting.,baseline_model,negative,0.862500
5,Feeling very overwhelmed right now. Everything...,baseline_model,negative,0.853554
6,I miss you so much it physically hurts.,baseline_model,negative,0.624761
7,Deeply disappointed but not surprised.,baseline_model,negative,0.820304
8,This is too good to be true — waiting for the ...,baseline_model,positive,0.962206
9,"Very proud of how far I’ve come, even if no on...",baseline_model,positive,0.979289


In [29]:
# OPTIONAL: Log synthetic test cases to W&B
wandb.log({
    "synthetic_tests": wandb.Table(dataframe=generated_df)
})

Error: You must call wandb.init() before wandb.log()